## Лаборатторная работа №3 (регрессия)

Подгрузим необходимые библиотеки

In [1]:
import numpy as np
import pandas as pd
from  matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

Подгрузим датасет

In [2]:
df = pd.read_csv('./autos.csv')

In [3]:
df.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


Датасет содержит следующие поля:
- `dateCrawled` - Дата сбора объяевления
- `name` - Название объявления
- `seller` - Продавец - частный или диллер (`privat`, `gewerblich`)
- `offerType` - Продажа или покупка (`Angebot`, `Gesuch`)
- `price` - Цена объявления
- `abtest` - ? (`test`, `control`)
- `vehicleType` - тип автомобиля (`nan`, `coupe`, `suv`, `kleinwagen`, `limousine`, `cabrio`, `bus`,`kombi`, `andere`)
- `yearOfRegistration` - Дата первой постановки на учет автомобиля
- `gearbox` - Тип коробки передач ручная или автомат (`nan`, `manuell`, `automatik`)
- `powerPS` - мощность двигателя в л.с.
- `model` - Модель автомобиля
- `kilometer` -  Показания одометра
- `monthOfRegistration` - месяц первой постановки на учет автомобиля
- `fuelType` - Тип топлива (`nan`, `benzin`, `diesel`, `lpg`, `andere`, `hybrid`, `cng`, `elektro`)
- `brand` - Марка
- `notRepairedDamage` - Имеет ли автомобиль неотремонтированные повреждения (`nan`, `ja`, `nein`)
- `dateCreated` -  Дата появления объявления
- `nrOfPictures` - Количество изображений в объявлении
- `postalCode` - Почтовый индекс
- `lastSeen` -  Последняя дата появления объявления

Выведем статистику по датасету

In [4]:
df.describe()

,price,yearOfRegistration,powerPS,kilometer,monthOfRegistration,nrOfPictures,postalCode
count,3.715280e+05,371528.000000,371528.000000,371528.000000,371528.000000,371528.0,371528.00000
mean,1.729514e+04,2004.577997,115.549477,125618.688228,5.734445,0.0,50820.66764
std,3.587954e+06,92.866598,192.139578,40112.337051,3.712412,0.0,25799.08247
min,0.000000e+00,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.00000
25%,1.150000e+03,1999.000000,70.000000,125000.000000,3.000000,0.0,30459.00000
50%,2.950000e+03,2003.000000,105.000000,150000.000000,6.000000,0.0,49610.00000
75%,7.200000e+03,2008.000000,150.000000,150000.000000,9.000000,0.0,71546.00000
max,2.147484e+09,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.00000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371528 entries, 0 to 371527
Data columns (total 20 columns):
dateCrawled            371528 non-null object
name                   371528 non-null object
seller                 371528 non-null object
offerType              371528 non-null object
price                  371528 non-null int64
abtest                 371528 non-null object
vehicleType            333659 non-null object
yearOfRegistration     371528 non-null int64
gearbox                351319 non-null object
powerPS                371528 non-null int64
model                  351044 non-null object
kilometer              371528 non-null int64
monthOfRegistration    371528 non-null int64
fuelType               338142 non-null object
brand                  371528 non-null object
notRepairedDamage      299468 non-null object
dateCreated            371528 non-null object
nrOfPictures           371528 non-null int64
postalCode             371528 non-null int64
lastSeen              

In [6]:
df.shape

(371528, 20)

Отфильтруем выбросы

In [7]:
df = df.loc[(df["price"] > 0) & (df["price"] < 20000)]
df = df.loc[(df["yearOfRegistration"] > 1900) & (df["yearOfRegistration"] < 2020)]

Исправим название бренда

In [8]:
df["brand"] = df["brand"].replace("rover", "land_rover")

Токенизируем категориальные признаки с применением One Hot Encoding

In [9]:
df = pd.get_dummies(df, columns=["seller", "offerType", "vehicleType", "gearbox", "model", 
                                "brand", "fuelType", "notRepairedDamage"])

Удалим неиспользуемые столбцы

In [10]:
df.drop(["dateCrawled", "name", "abtest", "dateCreated", "nrOfPictures", "postalCode", "lastSeen"],
        axis=1, inplace=True)

In [11]:
df.shape

(343199, 317)

Выполним нормализацию и стандартизацию признаков

In [12]:
scalers = {
            "price": MinMaxScaler(),
            "powerPS": StandardScaler(),
            "kilometer": StandardScaler(),
            "yearOfRegistration": MinMaxScaler(),
            "monthOfRegistration": MinMaxScaler()
          }

for column, scaler in scalers.items():
    scaler.fit(df[column].values.reshape(-1, 1))
    df[column] = scaler.transform(df[column].values.reshape(-1, 1))

Подготовим данные для обучения

In [13]:
X = df.drop("price", axis=1)
y = df["price"]

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Выполним обучение модели линейной регрессии с регуляризаторами $L_1$, $L_2$

In [15]:
reg = Lasso(alpha=0.0001)
reg.fit(X_train, y_train)
y_test_pred = reg.predict(X_test)
lasso_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
print('MAE: ', mean_absolute_error(y_test, y_test_pred))  
print('MSE: ', mean_squared_error(y_test, y_test_pred))  
print('RMSE:', lasso_rmse)

MAE:  0.10218567658500768
MSE:  0.021168871807360882
RMSE: 0.14549526386573855


In [16]:
reg = Ridge(alpha=0.0001)
reg.fit(X_train, y_train)
y_test_pred = reg.predict(X_test)
ridge_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
print('MAE: ', mean_absolute_error(y_test, y_test_pred))  
print('MSE: ', mean_squared_error(y_test, y_test_pred))  
print('RMSE:', ridge_rmse)

MAE:  0.0988146509785228
MSE:  0.02020126830835013
RMSE: 0.1421311658586889


In [17]:
print("Lasso: {:.4%}\nRidge: {:.4%}".format(lasso_rmse / df["price"].median(),
                                            ridge_rmse / df["price"].median()))

Lasso: 100.7833%
Ridge: 98.4530%


In [18]:
print("Lasso: {:.4%}\nRidge: {:.4%}".format(lasso_rmse / df["price"].mean(),
                                            ridge_rmse / df["price"].mean()))

Lasso: 64.0494%
Ridge: 62.5685%


In [19]:
print("Lasso: {:.4%}\nRidge: {:.4%}".format( lasso_rmse / df["price"].std(),
                                             ridge_rmse / df["price"].std()))

Lasso: 64.7134%
Ridge: 63.2171%


Как видно из результатов, линейная регрессия недостаточно хорошо справляется с предсказанием цены автомобиля и может использоваться для предсказания порядка цены.